In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
throws_df = pd.read_csv('../data/raw/all_games_0630.csv')
completed_throws =  throws_df[throws_df.turnover != 1]
completed_throws['lateral_distance'] = (completed_throws['thrower_x'] - completed_throws['receiver_x']).abs()
completed_throws['vertical_distance'] = completed_throws['receiver_y'] - completed_throws['thrower_y']
completed_throws['yards_to_middle'] = completed_throws['thrower_x'].abs() - completed_throws['receiver_x'].abs()

/var/folders/r4/cqt7s71n4k18qp94swd6_njr0000gn/T/ipykernel_49838/572196646.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  throws_df = pd.read_csv('../data/raw/all_games_0630.csv')
/var/folders/r4/cqt7s71n4k18qp94swd6_njr0000gn/T/ipykernel_49838/572196646.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  completed_throws['lateral_distance'] = (completed_throws['thrower_x'] - completed_throws['receiver_x']).abs()
/var/folders/r4/cqt7s71n4k18qp94swd6_njr0000gn/T/ipykernel_49838/572196646.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [4]:
completed_throws = completed_throws[['2023' in x for x in completed_throws.gameID]]

In [5]:
def get_functions_and_cols(prefix):
    foo = lambda x: (x, len(x), np.mean(x), sum(x))
    cols = [f'{prefix}_distances', 'count', f'{prefix}_average', f'{prefix}_total']
    return foo, cols


In [6]:
foo, cols = get_functions_and_cols('lateral')
lateral_df = pd.DataFrame([x for x in completed_throws.groupby('thrower')['lateral_distance'].apply(list).agg(foo)], columns=cols)
lateral_df['thrower'] = completed_throws.groupby('thrower')['lateral_distance'].apply(list).index
foo, cols = get_functions_and_cols('vertical')
vertical_df = pd.DataFrame([x for x in completed_throws.groupby('thrower')['vertical_distance'].apply(list).agg(lambda x: (x, len(x), np.mean(x), sum(x)))], columns=cols)
vertical_df['thrower'] = completed_throws.groupby('thrower')['vertical_distance'].apply(list).index

foo, cols = get_functions_and_cols('middle')
middle_df = pd.DataFrame([x for x in completed_throws.groupby('thrower')['yards_to_middle'].apply(list).agg(lambda x: (x, len(x), np.mean(x), sum(x)))], columns=cols)
middle_df['thrower'] = completed_throws.groupby('thrower')['yards_to_middle'].apply(list).index

thrower_df = pd.merge(lateral_df, vertical_df, how='inner', on=['thrower', 'count'])[['thrower', 'count', 'lateral_distances', 'lateral_average', 'lateral_total', 'vertical_distances', 'vertical_average', 'vertical_total']]
thrower_df = pd.merge(thrower_df, middle_df, how='inner', on=['thrower', 'count'])[['thrower', 'count', 'lateral_distances', 'lateral_average', 'lateral_total', 'vertical_distances', 'vertical_average', 'vertical_total', 'middle_distances', 'middle_average', 'middle_total']]

In [7]:
# thrower_df = thrower_df[thrower_df['count'] >= 20]

In [8]:
def get_thrower_team(row):
    thrower = completed_throws[(completed_throws.thrower == row.thrower) & completed_throws.is_home_team].home_teamID
    return thrower.iloc[0] if len(thrower) != 0 else completed_throws[(completed_throws.thrower == row.thrower) & ~completed_throws.is_home_team].away_teamID.iloc[0]

In [9]:
thrower_df['team'] = thrower_df.apply(get_thrower_team, axis=1)

In [10]:
shred_thrower_df = thrower_df[thrower_df.team == 'shred']

In [12]:
shred_thrower_df.columns

Index(['thrower', 'count', 'lateral_distances', 'lateral_average',
       'lateral_total', 'vertical_distances', 'vertical_average',
       'vertical_total', 'middle_distances', 'middle_average', 'middle_total',
       'team'],
      dtype='object')

In [15]:
shred_thrower_df.sort_values('count', ascending=False)[['thrower', 'count', 'lateral_average', 'vertical_total']]

,thrower,count,lateral_average,vertical_total
423,lyorgason,304,7.956711,3110.530
323,jmiller,283,8.867703,2437.600
593,sconnole,253,9.627470,2441.040
503,myorgason,224,9.006607,1426.300
306,jkerr,178,9.567978,1441.095
697,wselfridg,160,8.815313,633.550
208,elheureux,104,9.730962,1036.730
235,glindsley,102,8.839608,906.410
284,jduennebe,71,8.782254,532.610
156,cyorgason,70,8.317000,673.560


In [232]:
shred_thrower_df.sort_values('vertical_total', ascending=False)

,thrower,count,lateral_distances,lateral_average,lateral_total,vertical_distances,vertical_average,vertical_total,middle_distances,middle_average,middle_total,team
423,lyorgason,304,"[17.46, 3.1400000000000006, 15.280000000000001...",7.956711,2418.84,"[7.300000000000001, 3.6099999999999994, 3.1999...",10.232007,3110.530,"[9.96, -3.1400000000000006, -11.86, -3.1500000...",-1.481776,-450.46,shred
593,sconnole,253,"[11.110000000000001, 9.82, 7.500000000000001, ...",9.627470,2435.75,"[5.449999999999999, -1.5, -5.449999999999996, ...",9.648379,2441.040,"[-11.110000000000001, -2.3200000000000003, 7.5...",-0.234822,-59.41,shred
323,jmiller,283,"[6.5, 1.0899999999999999, 6.470000000000001, 1...",8.867703,2509.56,"[20.41, 13.300000000000004, 16.020000000000003...",8.613428,2437.600,"[3.6199999999999997, 1.0899999999999999, -6.47...",-0.341696,-96.70,shred
306,jkerr,178,"[2.25, 0.07000000000000028, 2.66, 20.38, 3.230...",9.567978,1703.10,"[-9.890000000000004, -7.700000000000003, 27.39...",8.096039,1441.095,"[-2.25, 0.07000000000000028, -2.66, 18.12, -3....",0.872247,155.26,shred
503,myorgason,224,"[12.25, 18.509999999999998, 9.870000000000001,...",9.006607,2017.48,"[10.069999999999997, 2.6499999999999986, 4.129...",6.367411,1426.300,"[-10.010000000000002, -16.25, 9.87000000000000...",-0.174821,-39.16,shred
208,elheureux,104,"[35.47, 7.57, 15.549999999999999, 5.2500000000...",9.730962,1012.02,"[0.7100000000000009, 29.72, 15.070000000000004...",9.968558,1036.730,"[16.950000000000003, -7.57, -15.54999999999999...",0.069423,7.22,shred
235,glindsley,102,"[8.93, 0.879999999999999, 6.61, 7.300000000000...",8.839608,901.64,"[-2.25, 17.930000000000007, 9.199999999999996,...",8.886373,906.410,"[-8.93, 0.879999999999999, 6.61, -7.3000000000...",-0.539804,-55.06,shred
217,esaunders,62,"[7.92, 4.92, 2.1499999999999986, 1.02000000000...",10.093871,625.82,"[9.060000000000002, 23.050000000000004, 60.32,...",14.562419,902.870,"[-7.92, -4.92, 2.1499999999999986, -1.02000000...",-0.944516,-58.56,shred
156,cyorgason,70,"[10.229999999999999, 13.43, 4.299999999999999,...",8.317000,582.19,"[-11.049999999999997, 5.390000000000001, 12.75...",9.622286,673.560,"[10.229999999999999, 13.43, -4.299999999999999...",1.970714,137.95,shred
697,wselfridg,160,"[10.440000000000001, 16.57, 1.3499999999999996...",8.815313,1410.45,"[-2.039999999999999, 4.43, 19.93, 2.3900000000...",3.959687,633.550,"[10.440000000000001, -16.57, -1.34999999999999...",0.046187,7.39,shred


In [233]:
shred_thrower_df.loc[323]

thrower                                                         jmiller
count                                                               283
lateral_distances     [6.5, 1.0899999999999999, 6.470000000000001, 1...
lateral_average                                                8.867703
lateral_total                                                   2509.56
vertical_distances    [20.41, 13.300000000000004, 16.020000000000003...
vertical_average                                               8.613428
vertical_total                                                   2437.6
middle_distances      [3.6199999999999997, 1.0899999999999999, -6.47...
middle_average                                                -0.341696
middle_total                                                      -96.7
team                                                              shred
Name: 323, dtype: object